# 🤖 04_deep_learning.ipynb  
## Neural Nets: MLP & LSTM

### 1. Objective  
Teach a **Multi-Layer Perceptron (MLP)** and a **Long Short-Term Memory (LSTM)** to predict daily returns.

> _Like first drawing a still picture (MLP), then making a short cartoon (LSTM) that remembers what happened a few frames ago._

---

### 2. MLP

- **Input:** today’s `rsi` and `macd`  
- **Architecture:** Dense(64) → Dropout → Dense(32) → Dense(1)  
- **Train:** 30 epochs, early stop if no improvement.  
- **Evaluate:** **MAE_mlp**, **R²_mlp**.

---

### 3. LSTM

- **Sequence length:** 5 days  
- **Input shape:** (5 days × 2 features)  
- **Architecture:** LSTM(50) → Dense(1)  
- **Train & evaluate** similarly.

---

### 4. Save Models & Test Data  
- Store `mlp_model/`, `lstm_model/` folders.  
- Save `X_test.npy`, `y_test.npy` (for MLP) and `X_te2.npy`, `y_te2.npy` (for LSTM).

## Reload features & split

In [1]:
import pandas as pd
import numpy as np
from pathlib import Path

root = Path().resolve().parent
df = pd.read_csv(root/"data"/"processed"/"features.csv",
                 index_col="Date", parse_dates=True)

X = df[["rsi","macd"]].values
y = df["return"].values

split = int(len(X)*0.8)
X_train, X_test = X[:split], X[split:]
y_train, y_test = y[:split], y[split:]


## Build and train an MLP

In [2]:
import tensorflow as tf
from tensorflow.keras import Sequential, layers
from sklearn.metrics import mean_absolute_error

# Define MLP
mlp = Sequential([
    layers.Input(shape=(2,)),
    layers.Dense(64, activation="relu"),
    layers.Dropout(0.2),
    layers.Dense(32, activation="relu"),
    layers.Dense(1)
])
mlp.compile(optimizer="adam", loss="mse", metrics=["mae"])

# Train
history = mlp.fit(
    X_train, y_train,
    validation_data=(X_test,y_test),
    epochs=30,
    callbacks=[tf.keras.callbacks.EarlyStopping(patience=5)]
)

# Evaluate
mae_mlp = mlp.evaluate(X_test, y_test, verbose=0)[1]
print(f"MLP MAE: {mae_mlp:.5f}")


Epoch 1/30
31/31 [==============================] - 1s 14ms/step - loss: 40.5287 - mae: 4.9176 - val_loss: 2.0718 - val_mae: 1.2130
Epoch 2/30
31/31 [==============================] - 0s 3ms/step - loss: 13.8775 - mae: 2.8594 - val_loss: 0.6053 - val_mae: 0.7178
Epoch 3/30
31/31 [==============================] - 0s 3ms/step - loss: 6.7604 - mae: 2.0186 - val_loss: 0.1105 - val_mae: 0.1793
Epoch 4/30
31/31 [==============================] - 0s 3ms/step - loss: 3.5795 - mae: 1.4408 - val_loss: 0.0584 - val_mae: 0.2184
Epoch 5/30
31/31 [==============================] - 0s 3ms/step - loss: 2.7172 - mae: 1.2814 - val_loss: 0.0332 - val_mae: 0.1494
Epoch 6/30
31/31 [==============================] - 0s 3ms/step - loss: 1.8716 - mae: 1.0353 - val_loss: 0.0592 - val_mae: 0.2122
Epoch 7/30
31/31 [==============================] - 0s 3ms/step - loss: 1.5846 - mae: 0.9579 - val_loss: 0.0317 - val_mae: 0.1245
Epoch 8/30
31/31 [==============================] - 0s 3ms/step - loss: 1.2141 - mae: 0

## Build and train an LSTM

In [3]:
# Create sequences of 5 days
timesteps = 5
def make_seq(X, y, t):
    xs, ys = [], []
    for i in range(len(X)-t):
        xs.append(X[i:i+t])
        ys.append(y[i+t])
    return np.array(xs), np.array(ys)

X_seq, y_seq = make_seq(X, y, timesteps)
split2 = int(len(X_seq)*0.8)
X_tr2, X_te2 = X_seq[:split2], X_seq[split2:]
y_tr2, y_te2 = y_seq[:split2], y_seq[split2:]

# Define LSTM
lstm = Sequential([
    layers.LSTM(50, input_shape=(timesteps,2)),
    layers.Dense(1)
])
lstm.compile("adam","mse",["mae"])

history2 = lstm.fit(
    X_tr2, y_tr2,
    validation_data=(X_te2,y_te2),
    epochs=30,
    callbacks=[tf.keras.callbacks.EarlyStopping(patience=5)]
)

mae_lstm = lstm.evaluate(X_te2, y_te2, verbose=0)[1]
print(f"LSTM MAE: {mae_lstm:.5f}")


Epoch 1/30
31/31 [==============================] - 3s 27ms/step - loss: 0.0310 - mae: 0.1096 - val_loss: 0.0088 - val_mae: 0.0656
Epoch 2/30
31/31 [==============================] - 0s 7ms/step - loss: 0.0023 - mae: 0.0347 - val_loss: 0.0037 - val_mae: 0.0379
Epoch 3/30
31/31 [==============================] - 0s 7ms/step - loss: 9.0481e-04 - mae: 0.0210 - val_loss: 0.0024 - val_mae: 0.0297
Epoch 4/30
31/31 [==============================] - 0s 9ms/step - loss: 5.3067e-04 - mae: 0.0165 - val_loss: 0.0016 - val_mae: 0.0239
Epoch 5/30
31/31 [==============================] - 0s 7ms/step - loss: 3.7877e-04 - mae: 0.0142 - val_loss: 0.0014 - val_mae: 0.0218
Epoch 6/30
31/31 [==============================] - 0s 7ms/step - loss: 2.9906e-04 - mae: 0.0130 - val_loss: 0.0011 - val_mae: 0.0193
Epoch 7/30
31/31 [==============================] - 0s 7ms/step - loss: 2.5599e-04 - mae: 0.0119 - val_loss: 9.1643e-04 - val_mae: 0.0179
Epoch 8/30
31/31 [==============================] - 0s 7ms/step -

In [7]:
# --- save deep models & test data ---
from pathlib import Path
import numpy as np

root      = Path().resolve().parent
model_dir = root / "models"
model_dir.mkdir(exist_ok=True)

# Save the MLP
mlp.save(model_dir / "mlp_model")

# Save the LSTM
lstm.save(model_dir / "lstm_model")

# Save test arrays
np.save(model_dir / "X_test.npy", X_test)
np.save(model_dir / "y_test.npy", y_test)
np.save(model_dir / "X_te2.npy",   X_te2)
np.save(model_dir / "y_te2.npy",   y_te2)

print("✅ MLP and LSTM saved to", model_dir)



INFO:tensorflow:Assets written to: C:\Users\Antho\OneDrive\Documentos\Santiago\Finance project\sp500_dl\models\mlp_model\assets


INFO:tensorflow:Assets written to: C:\Users\Antho\OneDrive\Documentos\Santiago\Finance project\sp500_dl\models\mlp_model\assets


INFO:tensorflow:Assets written to: C:\Users\Antho\OneDrive\Documentos\Santiago\Finance project\sp500_dl\models\lstm_model\assets


INFO:tensorflow:Assets written to: C:\Users\Antho\OneDrive\Documentos\Santiago\Finance project\sp500_dl\models\lstm_model\assets


✅ MLP and LSTM saved to C:\Users\Antho\OneDrive\Documentos\Santiago\Finance project\sp500_dl\models
